-sandbox
<img width = '500' src="https://databricks.com/wp-content/uploads/2021/06/db-pride-logo.svg" style="float: left: margin: 30px"/>        <img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 10px"/>                

<br>
> # Bronze para Silver

<br>
## 01 - Investidores
- Atualizaremos as pontuações de crédito dos investidores usando os dados macroeconômicos mais recentes recebidos do banco central.

### Objetivo:
* ler a partir da camada de bronze
* Criar e renomear colunas
* Calcular valores no driver usando objeto ***`Row`***
* Use ***`when`*** e ***`otherwise`*** para condições com Spark
* Combine os dados disponíveis em um DataFrame e no Driver com ***`lit`***
* Escreva no formato Delta

-sandbox
<h2 style = "color: red"> Informações: </h2>

<br>
- **tabelas de nível Silver:**
  - sua finalidade:
    - limpar, filtrar, desduplicar, ajustar tabelas de bronze para criar tabelas de base confiáveis
    - introduzir novos recursos de engenharia, potencialmente usando também resultados agregados
    - fornecer todas as transformações necessárias para gerar tabelas de camada gold
  - o que são e o que não são:
    - tabelas transformadas para usar como base para as tabelas finais
    - não são tabelas bronze, todo problema é corrigido, todo novo recurso é criado
    - não são tabelas gold, não há visualizações amplas

- **Objetivo:**
  - vamos usar dados de terceiros para atualizar a pontuação de crédito de investidor usando uma [métrica macroeconômica](https://en.wikipedia.org/wiki/Economic_indicator).
  - utilidade: sempre obter informações atualizadas sobre nossos investidores
  - descrever como utilizar único valor extraído de uma tabela (linha + lit)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas: 

<br>
- Ler os dados macroeconômicos de ** `macroDataPath` ** em um DataFrame.
- A fonte contém todos os dados macroeconômicos históricos. 
- Colete a linha com a última data para o driver e atribua-a a uma variável.
- Calcule a métrica usando os elementos desta linha.
  * A fórmula multiplicará várias colunas e as dividirá pelo PIB do país.
  * Fórmula da métrica = ***`(fedfunds X dexuseu X unrate X dgs10 X a191rl1q225sbea X indpro X bamlh0a0hym2) / "gdp"`***
- Trazer os investidores. Ler os dados de **`investmentsPath`** para um DataFrame.
- Retorna apenas os campos dos investidores **`id_do_investidor`**, **`nome`**, **`educação`**, **`pontuação_de_crédito`**, **`id_de_assinatura`**, **` pontuação_de_risco`** e **`fav_stocks`**.
- Atualize a coluna **`credit_score`** adicione a métrica que foi calculada. 
  * Regra: se a pontuação de crédito de um investidor for **`3`** e a métrica for **`0,2`**, então a nova pontuação de crédito será **`3,2`**.
- Há uma condição para as pontuações de crédito atualizadas: devem estar entre **`0`** e **`10`**. 
    * Certifique-se de que não excedam **`10`** ou caiam abaixo de **`0`**. 
    * Caso esteja acima de **`10`**, defina o valor para **`10`**. 
    * Caso esteja abaixo de **`0`**, defina o valor como **`0`**.
- Grave o resultado em **`targetDirectory`** como uma tabela Delta, usando o modo de substituição e o método de substituição de esquema.
- Retorne o DataFrame.

In [0]:
def func_investidores(spark, macroDataPath, investorsPath, targetDirectory):
  
  
  from pyspark.sql.functions import max, col, lit, when
  
  
  # Ler os dados macroeconomic a partir de `macroDataPath`.
  dfMacro = (spark.read
                  .format('delta')
                  .load(macroDataPath))
  
  
  # Coleta a linha com a data mais recente
  maxData = (dfMacro.select(max(col("date")))
                    .collect())

  
  linhaMacro = (dfMacro.filter(col("date") == maxData[0][0])
                       .collect()[0])
  
  
  # Calcule a métrica usando os elementos da linha acima, usando a fórmula abaixo:
  # Metric = (fedfunds X dexuseu X unrate X dgs10 X a191rl1q225sbea X indpro X bamlh0a0hym2) / gdp
  metrica = (linhaMacro["fedfunds"] * 
             linhaMacro["dexuseu"] * 
             linhaMacro["unrate"] * 
             linhaMacro["dgs10"] * 
             linhaMacro["a191rl1q225sbea"] * 
             linhaMacro["indpro"] * 
             linhaMacro["bamlh0a0hym2"] / 
             linhaMacro["gdp"])
  
  
  # Ler Investidores 
  # Retorne apenas os campos `investor_id`,` name`, `education`,` credit_score`, `subscription_id`,` risk_score` e `fav_stocks` de investidores.
  # Atualize a coluna `credit_score` adicionando a métrica que você calculou anteriormente.
  # Corrigir as pontuações de crédito: devem estar entre 0 e 10. 
    # Certifique-se de que não excedam 10 ou abaixo de 0. 
    # Caso esteja acima de 10, defina o valor para 10. Caso esteja abaixo de 0, defina o valor para 0 .
  dfInvestor = (spark.read
                      .format("delta")
                      .load(investorsPath)
                      .select("investor_id", 
                              "name", 
                              "education", 
                              "credit_score", 
                              "subscription_id", 
                              "risk_score", 
                              "fav_stocks")
                      .withColumn("credit_score", (col("credit_score") + lit(metrica)))
                      .withColumn("credit_score", when(col("credit_score") > 10, lit(10)).otherwise(col("credit_score")))
                      .withColumn("credit_score", when(col("credit_score") < 0, lit(0)).otherwise(col("credit_score"))))
  
  
  # Grave o resultado em targetDirectory como uma tabela Delta, usando o modo de substituição e o método de esquema de substituição.
  (dfInvestor.write
              .mode("overwrite")
              .format("delta")
              .option("overwriteSchema", True)
              .save(targetDirectory))
  
  
  # Retorne o DataFrame
  return dfInvestor

In [0]:
dfFinal = func_investidores(spark, macroDataPath, investorsPath, targetDirectory)

In [0]:
display(dfFinal)

investor_id,name,education,credit_score,subscription_id,risk_score,fav_stocks
0,Beverly Hardy,Some college but no degree,1.5610122744141655,1,2,"AKAM,ANDV,BDX,CHRW,DLTR,EXC,VRSK"
1,Shirley Goldberg,9th grade,1.5610122744141655,0,5,"EXR,F,KMB,KO,NKE,PYPL,STI,SYF,TXT"
2,Ivan Gonzales,Some college but no degree,2.5610122744141655,0,2,"BF.B,BLL,FISV,HLT,JPM,LB,MAC,PFE,ZTS"
3,Myrtle Cheek,Some college but no degree,10.0,0,9,"DISCA,FE,FISV,KMI,MCK,NTRS,PHM,RJF,WBA"
4,Judith Miller,10th grade,10.0,0,5,"ADM,COF,ECL,MNST,PFE,PRGO,PYPL,ROST,SRCL"
5,Nancy Anderson,Some college but no degree,2.5610122744141655,1,8,"CNP,DISCK,GD,SYK,TWX,WBA"
6,Margaret Johnson,Some college but no degree,1.5610122744141655,1,9,"AXP,COG,EW,ILMN,MSI,NOV,SNI,TPR"
7,Elizabeth Kelly,High school graduate,9.561012274414166,0,3,"DHI,ICE,LNT,MCK,MGM,RF"
8,Donald Henson,High school graduate,10.0,1,2,"C,CBS,CLX,COP,COST,HOG,ILMN,IPG,SBUX,SIG,WRK"
9,Amanda Lee,Bachelors degree(BA AB BS),3.5610122744141655,1,5,"AGN,AZO,GILD,MPC,PM,SNPS,TPR,UA,WDC"


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_investidores, spark, macroDataPath, investorsPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Silver table in place,
1,Silver table has correct content,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Investidores 2]($./02-Investidores 2)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>